# This notebook goes through the options for orthopyroxene-Liquid thermometry. 
- Opx-Liq matching is not covered in this tutorial, there is a separate folder "Opx_Liquid_melt_matching" for that 

## First, load the necessary python things

In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append("../..") # This allows you to put the python file in the folder above. So you can have lots of sub folders
import matplotlib.pyplot as plt
import Thermobar as pt
import sympy as sym

## Now, load the data 

In [2]:
out=pt.import_excel('Opx_Liq_Example.xlsx', sheet_name="Sheet1")
my_input=out['my_input']
Liqs=out['Liqs']
Opxs=out['Opxs']

C:\Users\penny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


We have set Fe3Fet_Liq to zero, as you only entered FeOt. You can input a Fe3FeT_Liq column to specify this value instead


# Working out options for thermometers/barometers
- At any point, you can do help(pt.function) to get some more information

## For example, here we get information on inputs/outputs for Cpx-Liq thermometry, including the equation options

In [3]:
help(pt.calculate_opx_liq_temp)

Help on function calculate_opx_liq_temp in module Thermobar.orthopyroxene_thermobarometry:

calculate_opx_liq_temp(*, equationT, opx_comps=None, liq_comps=None, meltmatch=None, P=None, eq_tests=False, Fe3FeT_Liq=None, H2O_Liq=None)
     Orthopyroxene-Liquid thermometer, user specifies equation,
     and calculates temperature in Kelvin.  Also has option to calculate equilibrium tests.
    
    Parameters
     -------
    
     opx_comps: DataFrame
         Orthopyroxene compositions with column headings SiO2_Opx, MgO_Opx etc.
    
     liq_comps: DataFrame
         Liquid compositions with column headings SiO2_Liq, MgO_Liq etc.
    
     meltmatch: DataFrame
         Combined Opx-Liquid compositions. Used for "melt match" functionality.
    
     EquationT: str
         Choice of equation:
         |  T_Opx_Beatt1993
         |  T_Put2008_eq28a
         |  T_Put2008_eq28b_opx_sat
    
    
     P: float, int, series, str  ("Solve")
         Pressure in kbar
         Only needed for P-s

C:\Users\penny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Alternatively, you can access the function names as well as the inputs like this:

In [4]:
pt.Opx_Liq_T_funcs

{<function Thermobar.orthopyroxene_thermobarometry.T_Opx_Beatt1993(P, *, CaO_Liq_cat_frac, FeOt_Liq_cat_frac, MgO_Liq_cat_frac, MnO_Liq_cat_frac, Al2O3_Liq_cat_frac, TiO2_Liq_cat_frac)>,
 <function Thermobar.orthopyroxene_thermobarometry.T_Put2008_eq28a(P, *, H2O_Liq, ln_Fm2Si2O6_liq, MgO_Liq_cat_frac, K2O_Liq_cat_frac, FeOt_Liq_cat_frac, FeOt_Opx_cat_6ox)>,
 <function Thermobar.orthopyroxene_thermobarometry.T_Put2008_eq28b_opx_sat(P, *, H2O_Liq, MgO_Liq_cat_frac, CaO_Liq_cat_frac, K2O_Liq_cat_frac, MnO_Liq_cat_frac, FeOt_Liq_cat_frac, FeOt_Opx_cat_6ox, Al2O3_Liq_cat_frac, TiO2_Liq_cat_frac, Mg_Number_Liq_NoFe3)>}

In [5]:
pt.Opx_Liq_P_funcs

{<function Thermobar.orthopyroxene_thermobarometry.P_Put2008_eq29a(T, *, SiO2_Liq_cat_frac, MgO_Liq_cat_frac, FeOt_Opx_cat_6ox, FmAl2SiO6, Na2O_Liq_cat_frac, Al2O3_Liq_cat_frac, K2O_Liq_cat_frac, H2O_Liq, NaAlSi2O6)>,
 <function Thermobar.orthopyroxene_thermobarometry.P_Put2008_eq29b(T, *, ln_FmAl2SiO6_liq, Al2O3_Liq_cat_frac, MgO_Liq_cat_frac, FeOt_Liq_cat_frac, SiO2_Opx_cat_6ox, FeOt_Opx_cat_6ox, Na2O_Liq_cat_frac, K2O_Liq_cat_frac, H2O_Liq)>,
 <function Thermobar.orthopyroxene_thermobarometry.P_Put_Felsic_Opx(T=None, *, Al2O3_Opx, Al2O3_Liq)>,
 <function Thermobar.orthopyroxene_thermobarometry.P_Put_Global_Opx(T=None, *, MgO_Liq, Al2O3_Opx, Al2O3_Liq, Na2O_Liq, K2O_Liq)>}

# Example calculation 1 - temperature for a known pressure and water content
- Here, we calculate temperature using the H2O content given in the H2O_Liq column (the default), and P=5 kbar
- There are a number of equations, but here we use T_Put2008_eq28a for temperature

In [6]:
Temp_T28a=pt.calculate_opx_liq_temp(opx_comps=Opxs, liq_comps=Liqs, equationT="T_Put2008_eq28a", P=5)
Temp_T28a

0    1391.122951
1    1391.135930
2    1335.249511
3    1350.916688
4    1311.016109
dtype: float64

## Or we can overwrite the water content in the function using an integer

In [7]:
Temp_T28a_0H2O=pt.calculate_opx_liq_temp(opx_comps=Opxs, liq_comps=Liqs, equationT="T_Put2008_eq28a", P=5, H2O_Liq=0)
Temp_T28a_0H2O

C:\Users\penny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    1451.178325
1    1492.564583
2    1426.377807
3    1444.872220
4    1397.863370
dtype: float64

## We can also specify eq_tests=True to get a full dataframe back with all the components, as well as a number of equilibrium test values

In [8]:
Temp_T28a_0H2O_EqTests=pt.calculate_opx_liq_temp(opx_comps=Opxs, liq_comps=Liqs, equationT="T_Put2008_eq28a", P=5, H2O_Liq=0, eq_tests=True)
Temp_T28a_0H2O_EqTests

C:\Users\penny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,T_K_calc,Eq_Test_Kd_Fe_Mg_Fet,Eq_Test_Kd_Fe_Mg_Fe2,SiO2_Liq,Kd Eq (Put2008+-0.06),TiO2_Liq,Al2O3_Liq,FeOt_Liq,MnO_Liq,MgO_Liq,...,Fm2Si2O6,En_Opx,Di_Opx,Mgno_OPX,ln_Fm2Si2O6_liq,ln_FmAl2SiO6_liq,Kd_Fe_Mg_Fet,Kd_Fe_Mg_Fe2,Ideal_Kd,Delta_Kd_Fe_Mg_Fe2
0,1451.178325,0.251582,0.251582,51.1,Y,0.93,17.5,8.91,0.18,6.09,...,0.957618,0.790815,0.028142,0.828850,5.211708,-2.879647,0.251582,0.251582,0.304877,0.053295
1,1492.564583,0.165104,0.165104,51.5,N,1.19,19.2,8.70,0.19,4.98,...,0.769186,0.660522,0.068126,0.860725,5.202722,3.931199,0.165104,0.165104,0.303256,0.138151
2,1426.377807,0.187634,0.187634,59.1,N,0.54,19.1,5.22,0.19,3.25,...,0.769585,0.656871,0.074962,0.855382,5.851735,4.175071,0.187634,0.187634,0.277184,0.089550
3,1444.872220,0.211297,0.211297,52.5,N,0.98,19.2,8.04,0.20,4.99,...,0.913443,0.764291,0.053125,0.839637,5.426741,1.519044,0.211297,0.211297,0.300632,0.089335
4,1397.863370,0.144506,0.144506,56.2,N,0.34,20.4,5.88,0.20,2.58,...,0.925110,0.777706,0.033103,0.844054,6.297377,2.489832,0.144506,0.144506,0.290284,0.145777


# Example calculation 2 - Calculating pressure for a known temperature

In [9]:
Temp_P29a=pt.calculate_opx_liq_press(opx_comps=Opxs, liq_comps=Liqs, equationP="P_Put2008_eq29a", T=1300)
Temp_P29a

C:\Users\penny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0     2.401637
1    12.237354
2     9.658047
3     5.585392
4     5.739632
dtype: float64

## Can do the same as before, overwrite water, ask for equilibrium tests. Using equation 30 from Putirka (2008) this time

In [10]:
Temp_P29a_0H2O=pt.calculate_opx_liq_press(opx_comps=Opxs, liq_comps=Liqs, equationP="P_Put2008_eq29a", T=1300, H2O_Liq=0, eq_tests=True)
Temp_P29a_0H2O

C:\Users\penny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,SiO2_Liq,P_kbar_calc,Eq_Test_Kd_Fe_Mg_Fet,Eq_Test_Kd_Fe_Mg_Fe2,Kd Eq (Put2008+-0.06),TiO2_Liq,Al2O3_Liq,FeOt_Liq,MnO_Liq,MgO_Liq,...,Fm2Si2O6,En_Opx,Di_Opx,Mgno_OPX,ln_Fm2Si2O6_liq,ln_FmAl2SiO6_liq,Kd_Fe_Mg_Fet,Kd_Fe_Mg_Fe2,Ideal_Kd,Delta_Kd_Fe_Mg_Fe2
0,51.1,-0.440763,0.251582,0.251582,Y,0.93,17.5,8.91,0.18,6.09,...,0.957618,0.790815,0.028142,0.828850,5.211708,-2.879647,0.251582,0.251582,0.304877,0.053295
1,51.5,7.599754,0.165104,0.165104,N,1.19,19.2,8.70,0.19,4.98,...,0.769186,0.660522,0.068126,0.860725,5.202722,3.931199,0.165104,0.165104,0.303256,0.138151
2,59.1,5.020447,0.187634,0.187634,N,0.54,19.1,5.22,0.19,3.25,...,0.769585,0.656871,0.074962,0.855382,5.851735,4.175071,0.187634,0.187634,0.277184,0.089550
3,52.5,0.947792,0.211297,0.211297,N,0.98,19.2,8.04,0.20,4.99,...,0.913443,0.764291,0.053125,0.839637,5.426741,1.519044,0.211297,0.211297,0.300632,0.089335
4,56.2,1.102032,0.144506,0.144506,N,0.34,20.4,5.88,0.20,2.58,...,0.925110,0.777706,0.033103,0.844054,6.297377,2.489832,0.144506,0.144506,0.290284,0.145777


## Can also specify a different Fe3FeT_Liq ratio, as delta Kd is calculated using just Fe2+ in the melt
- you can compare the equilibrium tests from Fet and Fe2 in the output columns. For the equilibrium test, we only use Fe2+

In [11]:
Temp_P29a_0H2O_30Fe=pt.calculate_opx_liq_press(opx_comps=Opxs, liq_comps=Liqs, equationP="P_Put2008_eq29a", T=1300, H2O_Liq=0, 
                                               eq_tests=True, Fe3FeT_Liq=0.3)
Temp_P29a_0H2O_30Fe

C:\Users\penny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,SiO2_Liq,P_kbar_calc,Eq_Test_Kd_Fe_Mg_Fet,Eq_Test_Kd_Fe_Mg_Fe2,Kd Eq (Put2008+-0.06),TiO2_Liq,Al2O3_Liq,FeOt_Liq,MnO_Liq,MgO_Liq,...,Fm2Si2O6,En_Opx,Di_Opx,Mgno_OPX,ln_Fm2Si2O6_liq,ln_FmAl2SiO6_liq,Kd_Fe_Mg_Fet,Kd_Fe_Mg_Fe2,Ideal_Kd,Delta_Kd_Fe_Mg_Fe2
0,51.1,-0.440763,0.251582,0.359403,Y,0.93,17.5,8.91,0.18,6.09,...,0.957618,0.790815,0.028142,0.828850,5.211708,-2.879647,0.251582,0.359403,0.304877,0.054526
1,51.5,7.599754,0.165104,0.235863,N,1.19,19.2,8.70,0.19,4.98,...,0.769186,0.660522,0.068126,0.860725,5.202722,3.931199,0.165104,0.235863,0.303256,0.067392
2,59.1,5.020447,0.187634,0.268049,Y,0.54,19.1,5.22,0.19,3.25,...,0.769585,0.656871,0.074962,0.855382,5.851735,4.175071,0.187634,0.268049,0.277184,0.009135
3,52.5,0.947792,0.211297,0.301853,Y,0.98,19.2,8.04,0.20,4.99,...,0.913443,0.764291,0.053125,0.839637,5.426741,1.519044,0.211297,0.301853,0.300632,0.001221
4,56.2,1.102032,0.144506,0.206437,N,0.34,20.4,5.88,0.20,2.58,...,0.925110,0.777706,0.033103,0.844054,6.297377,2.489832,0.144506,0.206437,0.290284,0.083846


# Example calculation 3 - Iterating pressure and temperature
- In reality, unles you are an experimentalist, you rarely know pressure and temperature
- In Keith Putirka's spreadsheets, you can link up columns to iterate P and T towards a solution, this can be done here using the function calculate_opx_liq_press_temp

## Here, we iterate equation 29b from Putirka (2008) for P, and Temp from Beattie (1993) for temperature
- Without specifying anything else, you get a dataframe with columns for calculated pressure and temperature

In [12]:
PT_iter_29b_Beatt=pt.calculate_opx_liq_press_temp(opx_comps=Opxs, liq_comps=Liqs, equationP="P_Put2008_eq29b", equationT="T_Opx_Beatt1993")
PT_iter_29b_Beatt

C:\Users\penny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,P_kbar_calc,T_K_calc
0,-2.083363,1396.863533
1,15.124832,1499.003345
2,15.723512,1496.583095
3,6.616612,1445.263817
4,6.404897,1400.627400


- Again, you can also specify equilibrium tests

In [14]:
PT_iter_29b_Beatt_EqTests=pt.calculate_opx_liq_press_temp(opx_comps=Opxs, liq_comps=Liqs, equationP="P_Put2008_eq29b", equationT="T_Opx_Beatt1993", eq_tests=True)
PT_iter_29b_Beatt_EqTests

C:\Users\penny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,P_kbar_calc,T_K_calc,SiO2_Liq,Eq_Test_Kd_Fe_Mg_Fet,Eq_Test_Kd_Fe_Mg_Fe2,Kd Eq (Put2008+-0.06),TiO2_Liq,Al2O3_Liq,FeOt_Liq,MnO_Liq,...,Fm2Si2O6,En_Opx,Di_Opx,Mgno_OPX,ln_Fm2Si2O6_liq,ln_FmAl2SiO6_liq,Kd_Fe_Mg_Fet,Kd_Fe_Mg_Fe2,Ideal_Kd,Delta_Kd_Fe_Mg_Fe2
0,-2.083363,1396.863533,51.1,0.251582,0.251582,Y,0.93,17.5,8.91,0.18,...,0.957618,0.790815,0.028142,0.828850,5.211708,-2.879647,0.251582,0.251582,0.304877,0.053295
1,15.124832,1499.003345,51.5,0.165104,0.165104,N,1.19,19.2,8.70,0.19,...,0.769186,0.660522,0.068126,0.860725,5.202722,3.931199,0.165104,0.165104,0.303256,0.138151
2,15.723512,1496.583095,59.1,0.187634,0.187634,N,0.54,19.1,5.22,0.19,...,0.769585,0.656871,0.074962,0.855382,5.851735,4.175071,0.187634,0.187634,0.277184,0.089550
3,6.616612,1445.263817,52.5,0.211297,0.211297,N,0.98,19.2,8.04,0.20,...,0.913443,0.764291,0.053125,0.839637,5.426741,1.519044,0.211297,0.211297,0.300632,0.089335
4,6.404897,1400.627400,56.2,0.144506,0.144506,N,0.34,20.4,5.88,0.20,...,0.925110,0.777706,0.033103,0.844054,6.297377,2.489832,0.144506,0.144506,0.290284,0.145777
